## Pyspark

In [1]:
from pyspark.sql import SparkSession
import findspark
findspark.init()

Spark = SparkSession.builder.master("local[*]").appName("SparkByExamples.com").getOrCreate()

AttributeError: 'Builder' object has no attribute 'appname'

## Pytorch

In [1]:
import torch
torch.cuda.is_available()


if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
    
print("using", device, "device")
gpu_device = torch.device("mps")

using cpu device


In [5]:
import time

matrix_size = 32*600

x = torch.randn(matrix_size, matrix_size)
y = torch.randn(matrix_size, matrix_size)

print("************ CPU SPEED ***************")
start = time.time()
result = torch.matmul(x,y)
print(time.time() - start)
print("verify device:", result.device)

x_gpu = x.to(gpu_device)
y_gpu = y.to(gpu_device)


for i in range(3):
    print("************ GPU SPEED ***************")
    start = time.time()
    result_gpu = torch.matmul(x_gpu,y_gpu)
    
    print(time.time() - start)
    print("verify device:", result_gpu.device)


************ CPU SPEED ***************
15.499061107635498
verify device: cpu
************ GPU SPEED ***************
0.05538320541381836
verify device: mps:0
************ GPU SPEED ***************
0.0014882087707519531
verify device: mps:0
************ GPU SPEED ***************
0.0001380443572998047
verify device: mps:0


In [7]:
result

tensor([[ -25.3012,  102.5469,  120.8521,  ...,   -4.4140,  113.6051,
         -153.2070],
        [ -62.4100,   53.4602,   25.9399,  ..., -180.3069,   10.4542,
          189.0947],
        [ -25.4482,  190.7199,  -45.3275,  ...,  206.1273, -335.8950,
         -120.9795],
        ...,
        [  31.2575,  195.0942,  -42.0568,  ...,   14.0960, -234.2721,
            4.6083],
        [  52.2986,  -72.9834,   57.5071,  ..., -204.1866,   43.3562,
          127.9590],
        [  65.7876,  186.1476,   93.6184,  ...,  -17.1774,  -38.8159,
          -25.4098]])

In [6]:
result_gpu

/Users/woj/Library/Python/3.9/lib/python/site-packages/torch/_tensor_str.py:115: UserWarning: The operator 'aten::nonzero' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:11.)
  nonzero_finite_vals = torch.masked_select(


tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='mps:0')

## Tensorflow 

In [8]:
import tensorflow as tf
import tensorflow_datasets as tfds
print("TensorFLow version:",tf.__version__)
print("Num GPUs Available:", len(tf.config.experimental.list_physical_devices('GPU')))
print("Num CPUs Available:", len(tf.config.experimental.list_physical_devices('CPU')))



TensorFLow version: 2.10.0
Num GPUs Available: 1
Num CPUs Available: 1


In [9]:
from tensorflow.python.client import device_lib
def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]
print(get_available_devices()) 

['/device:CPU:0', '/device:GPU:0']Metal device set to: 
Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2022-11-26 11:35:59.375763: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-26 11:35:59.376092: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [10]:
tf.test.is_built_with_cuda()

False

In [12]:
%%time
import tensorflow as tf
import tensorflow_datasets as tfds
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.config.list_physical_devices('GPU')
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label
batch_size = 128
ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(batch_size)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(batch_size)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, kernel_size=(3, 3),
                 activation='relu'),
  tf.keras.layers.Conv2D(64, kernel_size=(3, 3),
                 activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
#   tf.keras.layers.Dropout(0.25),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
#   tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy'],
)
model.fit(
    ds_train,
    epochs=12,
    validation_data=ds_test,
)

TensorFlow version: 2.10.0
Num GPUs Available:  1
Cause: Unable to locate the source code of <function normalize_img at 0x29304cd30>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: Unable to locate the source code of <function normalize_img at 0x29304cd30>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: Unable to locate the source code of <function normalize_img at 0x29304cd30>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Epoch 1/12


2022-11-26 11:44:24.004888: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


469/469 [==============================] - ETA: 0s - loss: 0.1540 - accuracy: 0.9549

2022-11-26 11:44:33.907654: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


469/469 [==============================] - 11s 20ms/step - loss: 0.1540 - accuracy: 0.9549 - val_loss: 0.0500 - val_accuracy: 0.9835
Epoch 2/12
469/469 [==============================] - 9s 20ms/step - loss: 0.0435 - accuracy: 0.9866 - val_loss: 0.0357 - val_accuracy: 0.9881
Epoch 3/12
469/469 [==============================] - 9s 19ms/step - loss: 0.0271 - accuracy: 0.9913 - val_loss: 0.0509 - val_accuracy: 0.9839
Epoch 4/12
469/469 [==============================] - 9s 19ms/step - loss: 0.0182 - accuracy: 0.9942 - val_loss: 0.0307 - val_accuracy: 0.9901
Epoch 5/12
469/469 [==============================] - 9s 19ms/step - loss: 0.0121 - accuracy: 0.9965 - val_loss: 0.0320 - val_accuracy: 0.9902
Epoch 6/12
469/469 [==============================] - 9s 19ms/step - loss: 0.0086 - accuracy: 0.9972 - val_loss: 0.0415 - val_accuracy: 0.9892
Epoch 7/12
469/469 [==============================] - 9s 20ms/step - loss: 0.0082 - accuracy: 0.9973 - val_loss: 0.0351 - val_accuracy: 0.9902
Epoch 8/1